# Data Mining and Discovery <br>
## Report Assignment
**Nazmul Hossain** | **23015862**

## Import necessary libraries

## Choose and load a random dataset from `datasets.txt`

## Dataset preprocessing

## Understand and apply subtopic 1 (Linear Regression) to the dataset

## Apply subtopic 2 (Classification)

## Compare and contrast both results (subtopic 1 vs subtopic 2)